In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [2]:
ps = nltk.PorterStemmer()
import pandas as pd

fullCorpus_F = pd.read_csv('C:/Users/Mohammad/Fake.csv')
fullCorpus_F=fullCorpus_F.iloc[:500]
fullCorpus_T = pd.read_csv('C:/Users/Mohammad/True.csv')
fullCorpus_T=fullCorpus_T.iloc[:500]
fullCorpus_FC= fullCorpus_F.drop(['subject', 'date','title'], axis=1)
fullCorpus_TC= fullCorpus_T.drop(['subject', 'date','title'], axis=1)
fullCorpus_FC.head(5)

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [3]:
fullCorpus_FC['label'] = 'False'
fullCorpus_TC['label'] = 'True'
fullCorpus_FC.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,False
1,House Intelligence Committee Chairman Devin Nu...,False
2,"On Friday, it was revealed that former Milwauk...",False
3,"On Christmas day, Donald Trump announced that ...",False
4,Pope Francis used his annual Christmas Day mes...,False


In [4]:
data=pd.concat([fullCorpus_FC, fullCorpus_TC], ignore_index=True)
data.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,False
1,House Intelligence Committee Chairman Devin Nu...,False
2,"On Friday, it was revealed that former Milwauk...",False
3,"On Christmas day, Donald Trump announced that ...",False
4,Pope Francis used his annual Christmas Day mes...,False


In [8]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)

#data['body_len'] = data['text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['text'])

X_features = pd.concat([data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,punct%,0,1,2,3,4,5,6,7,8,...,17493,17494,17495,17496,17497,17498,17499,17500,17501,17502
0,0.051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.049,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [13]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='False', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [14]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.98 / Accuracy: 0.99
Est: 10 / Depth: 20 ---- Precision: 0.953 / Recall: 1.0 / Accuracy: 0.975
Est: 10 / Depth: 30 ---- Precision: 0.981 / Recall: 1.0 / Accuracy: 0.99
Est: 10 / Depth: None ---- Precision: 0.935 / Recall: 1.0 / Accuracy: 0.965
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 1.0 / Accuracy: 1.0
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 1.0 / Accuracy: 1.0
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 1.0 / Accuracy: 1.0
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 1.0 / Accuracy: 1.0
Est: 100 / Depth: 10 ---- Precision: 0.99 / Recall: 1.0 / Accuracy: 0.995
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 1.0 / Accuracy: 1.0
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 1.0 / Accuracy: 1.0
Est: 100 / Depth: None ---- Precision: 0.99 / Recall: 1.0 / Accuracy: 0.995
